In [1]:
import tensorly as tl
from tensorly.decomposition import symmetric_power_iteration
import numpy as np
np.set_printoptions(precision=2, suppress=True)

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
  
def svd_whiten(X, k):
    # X: matrix to decompose
    # k: rank approximation
    
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    # rank k approximation
    U = U[:,:k]
    s = s[:k]
    Vt = Vt[:k,:]
    
    D_half = np.diag(np.sqrt(s))
    W = np.diag(1./np.sqrt(s)) @ U.T

    # so np.dot(np.dot(W.T, X),W) is the identity matrix
        
    return W , D_half, U

def rank_k_pseudoinverse(X, k):
    # pseudoinverse of rank k approximation of X
    U, s, Vt = np.linalg.svd(X, full_matrices=False)

    return np.linalg.pinv(U[:,:k] @ np.diag(s[:k]) @ Vt[:k,:])

def model_1():
    component_param =  {0: np.array([[0.15, 0.05, 0.8],
                                    [0.2, 0.15, 0.65],
                                    [0.12, 0.14, 0.74]]),
                      1: np.array([[0.2, 0.1  , 0.7],
                                   [0.1, 0.1, 0.8],
                                   [0.3, 0.2,  0.5]]),
                      2: np.array([[0.4, 0.15, 0.45],
                             [0.9, 0.03, 0.07],
                             [0.1, 0.7, 0.2]])}
    
    h_weights = np.array([0.2, 0.5, 0.3])
    return component_param, h_weights

def model_2():
    component_param =  {0: np.array([[0.15, 0.05, 0.2, .01, .59],
                                    [0.2, 0.15, .04, 0.55, .06],
                                    [0.12, 0.14, 0.7, .02, .02]]),
                      1: np.array([[0.2, 0.4  , 0.2, .1, .1],
                                   [0.1, 0.03, 0.57, .15, .15],
                                   [0.3, 0.1, .1, .33, .17]]),
                      2: np.array([[0.4, 0.45, .03, .02, .1],
                             [0.1, .22, .58, 0.03, 0.07],
                             [0.12, 0.38, .18, .12, .2]])}
    
    h_weights = np.array([0.2, 0.5, 0.3])
    return component_param, h_weights

def model_3():
    component_param =  {0: np.array([[.15, .05, .2 , .01, .59],
                                     [.2 , .15, .04, .55, .06],
                                     [.12, .14, .7 , .02, .02]]),
                        
                        1: np.array([[.2 , .4 , .2 , .1 , .1 ],
                                     [.1 , .03, .57, .15, .15],
                                     [.3 , .1 , .1 , .33, .17]]),
                        
                        2: np.array([[.4 , .45, .03, .02, .1 ],
                                     [.1 , .22, .07, .03, .58],
                                     [.12, .38, .18, .12, .2]]),
                        
                        3: np.array([[.12, .23, .43, .05, .17],
                                     [.44, .38, .06, .06, .06],
                                     [.21, .21, .21, .12, .25]]),
                        
                        4: np.array([[.38, .13, .32, .12, .05],
                                     [.30, .1 , .14, .31, .15],
                                     [.08, .04, .6 , .17, .11]])                          
                       }
    
    h_weights = np.array([0.15, 0.43, 0.21, .07, .14])
    return component_param, h_weights

def model_4():
    component_param =  {0: np.array([[.15, .05, .2 , .01, .23, .22, .14],
                                     [.2 , .15, .04, .35, .06, .15, .05],
                                     [.12, .14, .3 , .02, .02, .22, .18]]),
                        
                        1: np.array([[.23, .01, .22 ,.03 ,.31 ,.13, .07],
                                     [.1 , .03, .3 , .15, .15, .12, .15],
                                     [.3 , .1 , .1 , .17, .17, .07, .09]]),
                        
                        2: np.array([[.4 , .23, .03, .02, .1 , .02, .2 ],
                                     [.1 , .22, .07, .03, .56, .01, .01],
                                     [.12, .31, .18, .12, .2 , .03, .04]]),
                        
                        3: np.array([[.12, .13, .22, .05, .17, .1 , .21 ],
                                     [.44, .13, .06, .02, .06, .25, .04],
                                     [.08, .21, .21, .12, .21, .04, .13]]),
                        
                        4: np.array([[.29, .13, .26, .12, .05, .09, .06],
                                     [.19, .1 , .05, .31, .15, .11, .09],
                                     [.08, .04, .54, .11, .11, .06, .06]])                          
                       }
    
    h_weights = np.array([0.15, 0.3, 0.21, .2, .14])
    return component_param, h_weights

def generate_data_fixed(data_size=10000):
    # num_components = num_rows = num_views = 3
    
    component_param, h_weights = model_4()
    
    comp_label = np.arange(0,len(h_weights))  # there are num_components of components; zero-indexing
    observation_label = np.arange(0, len(component_param[0][0]))
    
    data = []
    for i in range(data_size):
        # choose a component
        choose = np.random.choice(comp_label, p = h_weights)
        datum = []
        for v in range(3):
            datum.append(np.random.choice(observation_label, p = component_param[choose][v]))            
        data.append(tuple(datum))
        
    rank = len(h_weights)
    obs_size = len(component_param[0][0])
    return  rank, obs_size, component_param, h_weights, data

In [2]:
data_len = 300000
rank, obs_size, cp, h, data = generate_data_fixed(data_size=data_len)
identity = np.eye(obs_size)
get_identity_col = lambda col_num: identity[:, col_num]
e0 = get_identity_col(0); e1 = get_identity_col(1); e2 = get_identity_col(2);

def create_view_vecs(data, view):
    return np.column_stack([get_identity_col(datum[view]) for datum in data])

In [3]:
view0s = create_view_vecs(data,0)
view1s = create_view_vecs(data,1)
view2s = create_view_vecs(data,2)

M01 = (view0s @ view1s.T)/len(data)
M02 = (view0s @ view2s.T)/len(data)
M10 = (view1s @ view0s.T)/len(data)
M12 = (view1s @ view2s.T)/len(data) 
M20 = (view2s @ view0s.T)/len(data)
M21 = (view2s @ view1s.T)/len(data)


M01_pinv = np.linalg.pinv(M01)
M02_pinv = np.linalg.pinv(M02)
M10_pinv = np.linalg.pinv(M10)
M12_pinv = np.linalg.pinv(M12)
M20_pinv = np.linalg.pinv(M20)
M21_pinv = np.linalg.pinv(M21)


C_0_to_2 = M21 @ M01_pinv
C_1_to_2 = M20 @ M10_pinv
C_2_to_1 = M10 @ M20_pinv
C_2_to_0 = M01 @ M21_pinv

M_2 = 0.5* (C_0_to_2 @ M01 @ C_1_to_2.T + C_1_to_2 @ M10 @ C_0_to_2.T)
np.sum(M_2)

0.9999999999999758

In [4]:
M_2.shape

(7, 7)

In [5]:
def get_all_3rd_order_tensor(view0s, view1s, view2s, obs_size):
    # get all the dummy estimate (before view transform) for all 3rd-order tensors
    M3 = {(i,j,k):np.zeros((obs_size, obs_size, obs_size)) if i != j and i != k and j != k else None \
                                    for i in range(3)\
                                    for j in range(3)\
                                    for k in range(3)}
    M3 = {key:value for key, value in M3.items() if M3[key] is not None}
    views = [view0s, view1s, view2s]
    for key in M3.keys():
        for i in range(data_len):
            M3[key] += views[key[0]][:,i][:,None,None] * views[key[1]][:,i][None,:,None] * views[key[2]][:,i][None,None,:]
        M3[key] /= data_len
    return M3

M3 = get_all_3rd_order_tensor(view0s, view1s, view2s, obs_size)
M012_trans = tl.tenalg.multi_mode_dot(M3[0,1,2], [C_0_to_2, C_1_to_2], modes=[0,1])
M021_trans = tl.tenalg.multi_mode_dot(M3[0,2,1], [C_0_to_2, C_1_to_2], modes=[0,2])
M102_trans = tl.tenalg.multi_mode_dot(M3[1,0,2], [C_1_to_2, C_0_to_2], modes=[0,1])
M120_trans = tl.tenalg.multi_mode_dot(M3[1,2,0], [C_1_to_2, C_0_to_2], modes=[0,2])
M201_trans = tl.tenalg.multi_mode_dot(M3[2,0,1], [C_0_to_2, C_1_to_2], modes=[1,2])
M210_trans = tl.tenalg.multi_mode_dot(M3[2,1,0], [C_1_to_2, C_0_to_2], modes=[1,2])

M_3 = (M012_trans+M021_trans+M102_trans+M120_trans+M201_trans+M210_trans)/6
np.sum(M_3)

0.9999999999999758

In [6]:
W, D_half, U = svd_whiten(M_2, rank) # a bit a cheating here by using rank variable
matprint( W @ M_2 @ W.T)
whitened_t01 = tl.tenalg.multi_mode_dot(M_2, [W, W], modes=[0,1])
whitened_t012 = tl.tenalg.multi_mode_dot(M_3, [W, W, W], modes=[0,1,2])

           1   4.41485e-16   1.35297e-16  5.41796e-16  3.66322e-15  
-2.75143e-17             1    3.4363e-19  1.52511e-16  2.90856e-15  
-1.22549e-16   1.99304e-16             1   7.4349e-17   -8.036e-15  
 1.63776e-16   1.78738e-16  -6.52861e-16            1  5.84756e-15  
 1.39818e-15  -8.00979e-16   6.92983e-17  7.38637e-17            1  


In [7]:
w = []
u = []
deflated = None
for i in range(rank):
    if deflated is None:        
        w_temp, u_temp, deflated = symmetric_power_iteration(whitened_t012, n_repeat=50, n_iteration=40)        
    else:
        w_temp, u_temp, deflated = symmetric_power_iteration(deflated, n_repeat=50, n_iteration=40)    
    w.append(w_temp)
    u.append(u_temp)

In [8]:
recovered_h = np.array([pow(1/i,2) for i in w])
os = [w[index] * U @ D_half @ u_i for index, u_i in enumerate(u)]
for index, os_i in enumerate(os):
    print("component weight: ", round(recovered_h[index], 3))
    print("first view: ", C_2_to_0 @ os_i )
    print("second view:", C_2_to_1 @ os_i )
    print("third view: ", os_i)
    print(" ")    

component weight:  0.006
first view:  [ 0.95  0.58 -0.59 -0.17 -0.17  0.12  0.35]
second view: [ 0.56  0.24 -0.29  0.7  -0.39  0.28 -0.04]
third view:  [-0.35  0.71  0.57 -0.36 -0.31  0.53  0.27]
 
component weight:  0.017
first view:  [-0.38 -0.02  0.39  0.09  0.14  0.28  0.26]
second view: [ 0.66  0.23 -0.32  0.14 -0.26  0.43 -0.11]
third view:  [-0.13 -0.    0.38 -0.03  0.11  0.16  0.27]
 
component weight:  0.086
first view:  [0.14 0.04 0.25 0.16 0.04 0.12 0.1 ]
second view: [0.14 0.04 0.06 0.34 0.07 0.1  0.11]
third view:  [ 0.04 -0.03  0.62  0.1   0.05  0.02  0.03]
 
component weight:  0.143
first view:  [ 0.14 -0.04  0.21  0.01  0.27  0.04 -0.04]
second view: [-0.14  0.02  0.32 -0.01  0.35 -0.05  0.12]
third view:  [ 0.34  0.01 -0.1   0.19  0.18 -0.02  0.01]
 
component weight:  0.459
first view:  [-0.    0.02 -0.17 -0.04 -0.11 -0.06 -0.03]
second view: [-0.06 -0.04 -0.08 -0.04 -0.11 -0.05 -0.04]
third view:  [-0.11  0.01 -0.1  -0.08 -0.09 -0.   -0.04]
 


In [9]:
def process_os_i(vec):
    processed = [i if i > 0 else 0.01 for i in vec]
    return np.array(processed)/sum(processed)

recovered_h = process_os_i(np.array([pow(1/i,2) for i in w]))
os = [w[index] * U @ D_half @ u_i for index, u_i in enumerate(u)]
for index, os_i in enumerate(os):
    print("component weight: ", round(recovered_h[index], 3))
    print("first view: ", process_os_i(C_2_to_0 @ os_i))
    print("second view:", process_os_i(C_2_to_1 @ os_i) )
    print("third view: ", process_os_i(os_i))
    print(" ")    

component weight:  0.008
first view:  [0.47 0.29 0.   0.   0.   0.06 0.17]
second view: [0.31 0.13 0.01 0.39 0.01 0.15 0.01]
third view:  [0.   0.34 0.27 0.   0.   0.25 0.13]
 
component weight:  0.024
first view:  [0.01 0.01 0.33 0.07 0.12 0.24 0.22]
second view: [0.44 0.15 0.01 0.09 0.01 0.29 0.01]
third view:  [0.01 0.01 0.4  0.01 0.12 0.16 0.29]
 
component weight:  0.121
first view:  [0.17 0.05 0.29 0.19 0.04 0.14 0.11]
second view: [0.16 0.05 0.07 0.4  0.08 0.11 0.13]
third view:  [0.05 0.01 0.7  0.12 0.06 0.03 0.03]
 
component weight:  0.201
first view:  [0.21 0.01 0.3  0.02 0.38 0.06 0.01]
second view: [0.01 0.02 0.38 0.01 0.42 0.01 0.14]
third view:  [0.45 0.02 0.01 0.25 0.24 0.01 0.01]
 
component weight:  0.646
first view:  [0.13 0.21 0.13 0.13 0.13 0.13 0.13]
second view: [0.14 0.14 0.14 0.14 0.14 0.14 0.14]
third view:  [0.15 0.11 0.15 0.15 0.15 0.15 0.15]
 


In [10]:
M01_pinv = rank_k_pseudoinverse(M01, rank)
M02_pinv = rank_k_pseudoinverse(M02, rank)
M10_pinv = rank_k_pseudoinverse(M10, rank)
M12_pinv = rank_k_pseudoinverse(M12, rank)
M20_pinv = rank_k_pseudoinverse(M20, rank)
M21_pinv = rank_k_pseudoinverse(M21, rank)

T = M3[0,1,2]
for _ in range(rank):
    u = np.random.rand(obs_size)
    u = np.divide(u, np.linalg.norm(u))
  
    for _ in range(80):
        u = tl.tenalg.multi_mode_dot(T, [M01_pinv @ u, M02_pinv @ u], modes=[1,2])
        u = np.divide(u, np.linalg.norm(u))
    
    a = np.divide(u, np.sqrt(np.abs(np.dot(u, M10_pinv @ M12 @ M02_pinv @ u))))
    b = M12 @ M02_pinv @ a
    c = M21 @ M01_pinv @ a
    
    lambda_1 =  tl.tenalg.multi_mode_dot(T, [ M10_pinv @ M12 @ M02_pinv @ a, M01_pinv @ a, M02_pinv @ a], modes=[0,1,2])
    T = T - abs(lambda_1)*np.tensordot(np.tensordot(a,b,axes=0),c,axes=0) # deflation
    
    print("component weight: ", round(pow(1/lambda_1,2),3))
    print("first view: ", a/np.sum(a))
    print("second view:", b/np.sum(b))
    print("third view: ", c/np.sum(c))
    print(" ")

component weight:  0.2
first view:  [0.13 0.14 0.23 0.04 0.15 0.1  0.21]
second view: [0.44 0.13 0.06 0.03 0.02 0.27 0.05]
third view:  [0.06 0.21 0.23 0.12 0.2  0.05 0.13]
 
component weight:  0.11
first view:  [ 0.1   0.07  0.16 -0.03  0.27  0.26  0.16]
second view: [ 0.25  0.19 -0.02  0.36  0.05  0.16  0.01]
third view:  [0.11 0.12 0.24 0.   0.07 0.24 0.22]
 
component weight:  0.342
first view:  [0.24 0.01 0.22 0.03 0.3  0.14 0.07]
second view: [0.08 0.04 0.28 0.19 0.17 0.1  0.14]
third view:  [0.29 0.09 0.13 0.16 0.16 0.08 0.09]
 
component weight:  0.22
first view:  [0.41 0.22 0.04 0.02 0.11 0.01 0.19]
second view: [ 0.1   0.21  0.1  -0.01  0.59 -0.    0.02]
third view:  [0.16 0.31 0.13 0.14 0.22 0.01 0.03]
 
component weight:  0.125
first view:  [0.25 0.11 0.28 0.1  0.07 0.12 0.06]
second view: [0.19 0.1  0.03 0.38 0.08 0.13 0.09]
third view:  [0.07 0.06 0.6  0.07 0.02 0.1  0.08]
 


In [164]:
model_4()

({0: array([[0.15, 0.05, 0.2 , 0.01, 0.23, 0.22, 0.14],
         [0.2 , 0.15, 0.04, 0.35, 0.06, 0.15, 0.05],
         [0.12, 0.14, 0.3 , 0.02, 0.02, 0.22, 0.18]]),
  1: array([[0.23, 0.01, 0.22, 0.03, 0.31, 0.13, 0.07],
         [0.1 , 0.03, 0.3 , 0.15, 0.15, 0.12, 0.15],
         [0.3 , 0.1 , 0.1 , 0.17, 0.17, 0.07, 0.09]]),
  2: array([[0.4 , 0.23, 0.03, 0.02, 0.1 , 0.02, 0.2 ],
         [0.1 , 0.22, 0.07, 0.03, 0.56, 0.01, 0.01],
         [0.12, 0.31, 0.18, 0.12, 0.2 , 0.03, 0.04]]),
  3: array([[0.12, 0.13, 0.22, 0.05, 0.17, 0.1 , 0.21],
         [0.44, 0.13, 0.06, 0.02, 0.06, 0.25, 0.04],
         [0.08, 0.21, 0.21, 0.12, 0.21, 0.04, 0.13]]),
  4: array([[0.29, 0.13, 0.26, 0.12, 0.05, 0.09, 0.06],
         [0.19, 0.1 , 0.05, 0.31, 0.15, 0.11, 0.09],
         [0.08, 0.04, 0.54, 0.11, 0.11, 0.06, 0.06]])},
 array([0.15, 0.3 , 0.21, 0.2 , 0.14]))

In [165]:
# so it seemed that iterate asymmetrically is more stable

In [169]:
105-20+1

86